In [2]:
from pathlib import Path
from typing import List, Tuple
import pandas as pd
import numpy as np


ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]

DATA = ROOT / "DATA"
CHAN = ROOT / "chan_data"
SEG_FAST = CHAN / "segments" / "fast"
OUT_BASE = CHAN / "mapped" / "segments_fast"

OUT_BASE.mkdir(parents=True, exist_ok=True)

symbols = ["BTCUSDT", "ETHUSDT"]
tfs = ["5m", "15m", "30m", "1h", "4h"]


def _read_candles(p: Path) -> pd.DataFrame:

    df = pd.read_csv(p)
    if "timestamp" not in df.columns:
        for c in ["time", "datetime", "ts"]:
            if c in df.columns:
                df = df.rename(columns={c: "timestamp"})
                break
    need = {"timestamp", "open", "high", "low", "close"}
    miss = need - set(df.columns)
    if miss:
        raise ValueError(f"The K line lacks the necessary column: {sorted(miss)} @ {p}")
    return df


def prepare_candles(df: pd.DataFrame, tz: str = "UTC") -> pd.DataFrame:

    out = df.copy()
    out["timestamp"] = pd.to_datetime(out["timestamp"], utc=True, errors="coerce")
    out = out.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)
    out = out.drop_duplicates(subset=["timestamp"], keep="last").reset_index(drop=True)
    out["bar_index"] = np.arange(len(out), dtype=int)
    return out


def _read_segments(p: Path) -> pd.DataFrame:

    df = pd.read_csv(p, parse_dates=["timestamp_start", "timestamp_end"])
    need = {"segment_index", "timestamp_start", "timestamp_end"}
    miss = need - set(df.columns)
    if miss:
        raise ValueError(f"The segment lacks the necessary columns: {sorted(miss)} @ {p}")
    df["timestamp_start"] = pd.to_datetime(df["timestamp_start"], utc=True)
    df["timestamp_end"] = pd.to_datetime(df["timestamp_end"], utc=True)
    df = df.sort_values(["timestamp_start", "timestamp_end"]).reset_index(drop=True)
    return df



def map_segments_to_candles(
    segments: pd.DataFrame,
    candles: pd.DataFrame,
    ts_start_col: str = "timestamp_start",
    ts_end_col: str = "timestamp_end",
    seg_index_col: str = "segment_index",
) -> pd.DataFrame:

    if segments is None or segments.empty:
        return pd.DataFrame(columns=list(segments.columns) + ["start_bar_index", "end_bar_index"])
    if candles is None or candles.empty:
        raise ValueError("candles It is empty and cannot be mapped.。")

    seg = segments.copy().reset_index(drop=True)
    c = candles.copy().reset_index(drop=True)

    ts = c["timestamp"].to_numpy()

    svals = seg[ts_start_col].to_numpy()
    evals = seg[ts_end_col].to_numpy()

    start_idx = np.searchsorted(ts, svals, side="left")
    end_idx = np.searchsorted(ts, evals, side="right") - 1

    start_idx = np.clip(start_idx, 0, len(ts) - 1)
    end_idx = np.clip(end_idx,   0, len(ts) - 1)

    bad = start_idx > end_idx
    end_idx[bad] = start_idx[bad]

    out = seg.copy()
    out["start_bar_index"] = start_idx.astype("int64")
    out["end_bar_index"] = end_idx.astype("int64")

    # Check: Make sure that the end_bar_index time is no later than segment_end (avoid future functions)
    # (Tolerance equals; if it is later, retreat to the suitable position)
    end_ts = c.loc[out["end_bar_index"].astype(int), "timestamp"].to_numpy()
    too_late = end_ts > evals
    if np.any(too_late):
        fix_idx = out.loc[too_late, "end_bar_index"].astype(int).to_numpy()
        fix_idx -= 1
        fix_idx = np.clip(fix_idx, 0, len(ts) - 1)
        out.loc[too_late, "end_bar_index"] = fix_idx.astype("int64")

    # start <= end
    bad2 = out["start_bar_index"] > out["end_bar_index"]
    if bad2.any():
        out.loc[bad2, "end_bar_index"] = out.loc[bad2, "start_bar_index"]

    return out


def save_mapped_segments(
    out_dir: Path,
    sym: str,
    tf: str,
    mapped_df: pd.DataFrame
) -> Path:
    d = out_dir / sym
    d.mkdir(parents=True, exist_ok=True)
    p = d / f"{tf}_segments_mapped.csv"
    mapped_df.to_csv(p, index=False)
    return p


def quick_check(mapped_df: pd.DataFrame, candles: pd.DataFrame, sample_n: int = 3) -> List[Tuple[int, int, str]]:

    if mapped_df.empty:
        return []
    rng = np.random.default_rng(123)
    idxs = rng.choice(len(mapped_df), size=min(sample_n, len(mapped_df)), replace=False)
    res = []
    for i in idxs:
        r = mapped_df.iloc[i]
        si, ei = int(r["start_bar_index"]), int(r["end_bar_index"])
        if 0 <= si <= ei < len(candles):
            verdict = "OK"
        else:
            verdict = "OUT_OF_RANGE"
        res.append((si, ei, verdict))
    return res



rows_log = []
for sym in symbols:
    for tf in tfs:
        p_candles = DATA / sym / f"{tf}.csv"
        p_seg = SEG_FAST / sym / f"{tf}_segments.csv"

        if not p_candles.exists():
            print(f"[skip] {sym}-{tf}: candles missing -> {p_candles}")
            continue
        if not p_seg.exists():
            print(f"[skip] {sym}-{tf}: segments missing -> {p_seg}")
            continue

        print(f"[run] {sym}-{tf}")
        candles = prepare_candles(_read_candles(p_candles))
        seg = _read_segments(p_seg)

        mapped = map_segments_to_candles(seg, candles)

        n = len(mapped)
        bad = int((mapped["start_bar_index"] > mapped["end_bar_index"]).sum())
        out_p = save_mapped_segments(OUT_BASE, sym, tf, mapped)
        probe = quick_check(mapped, candles, sample_n=3)

        rows_log.append(dict(
            symbol=sym, tf=tf, n_segments=n,
            n_bad=bad, out=str(out_p), sample=str(probe)
        ))
        print(f"  [saved] n={n}, bad={bad}, sample={probe}")

print("[done] all segments mapped & saved.")


log_df = pd.DataFrame(rows_log)
display(log_df)

[run] BTCUSDT-5m
  [saved] n=112, bad=0, sample=[(22823, 23906, 'OK'), (25903, 25959, 'OK'), (509, 528, 'OK')]
[run] BTCUSDT-15m
  [saved] n=116, bad=0, sample=[(9890, 10152, 'OK'), (11567, 11599, 'OK'), (218, 276, 'OK')]
[run] BTCUSDT-30m
  [saved] n=105, bad=0, sample=[(5493, 5625, 'OK'), (6224, 6239, 'OK'), (47, 62, 'OK')]
[run] BTCUSDT-1h
  [saved] n=64, bad=0, sample=[(2457, 2578, 'OK'), (2923, 3012, 'OK'), (12, 69, 'OK')]
[run] BTCUSDT-4h
  [saved] n=17, bad=0, sample=[(1132, 1169, 'OK'), (834, 847, 'OK'), (12, 23, 'OK')]
[run] ETHUSDT-5m
  [saved] n=408, bad=0, sample=[(30990, 31067, 'OK'), (34130, 34153, 'OK'), (1138, 1223, 'OK')]
[run] ETHUSDT-15m
  [saved] n=256, bad=0, sample=[(11426, 11442, 'OK'), (12617, 12664, 'OK'), (322, 471, 'OK')]
[run] ETHUSDT-30m
  [saved] n=142, bad=0, sample=[(5654, 5701, 'OK'), (6420, 6444, 'OK'), (109, 187, 'OK')]
[run] ETHUSDT-1h
  [saved] n=78, bad=0, sample=[(2581, 2611, 'OK'), (2924, 2961, 'OK'), (123, 139, 'OK')]
[run] ETHUSDT-4h
  [saved] 

symbol   tf  n_segments  n_bad  \
0  BTCUSDT   5m         112      0   
1  BTCUSDT  15m         116      0   
2  BTCUSDT  30m         105      0   
3  BTCUSDT   1h          64      0   
4  BTCUSDT   4h          17      0   
5  ETHUSDT   5m         408      0   
6  ETHUSDT  15m         256      0   
7  ETHUSDT  30m         142      0   
8  ETHUSDT   1h          78      0   
9  ETHUSDT   4h          20      0   

                                                 out  \
0  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
1  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
2  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
3  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
4  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
5  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
6  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
7  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
8  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   
9  /Users/cc/PycharmProjects/ChanLab/chan_data/ma...   

                                              sample  
0  [(22823, 23906, 'OK'), (25903, 25959, 'OK'), (...  
1  [(9890, 10152, 'OK'), (11567, 11599, 'OK'), (2...  
2  [(5493, 5625, 'OK'), (6224, 6239, 'OK'), (47, ...  
3  [(2457, 2578, 'OK'), (2923, 3012, 'OK'), (12, ...  
4  [(1132, 1169, 'OK'), (834, 847, 'OK'), (12, 23...  
5  [(30990, 31067, 'OK'), (34130, 34153, 'OK'), (...  
6  [(11426, 11442, 'OK'), (12617, 12664, 'OK'), (...  
7  [(5654, 5701, 'OK'), (6420, 6444, 'OK'), (109,...  
8  [(2581, 2611, 'OK'), (2924, 2961, 'OK'), (123,...  
9  [(669, 706, 'OK'), (746, 762, 'OK'), (19, 30, ...

In [3]:

from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]

DATA = ROOT / "DATA"
MAP = ROOT / "chan_data" / "mapped" / "segments_fast"

symbols = ["BTCUSDT", "ETHUSDT"]
tfs = ["5m","15m","30m","1h","4h"]

def _read_candles(p):
    df = pd.read_csv(p)
    if "timestamp" not in df.columns:
        for c in ["time","datetime","ts"]:
            if c in df.columns:
                df = df.rename(columns={c: "timestamp"})
                break
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
    df = df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)
    return df

def _quick_checks(sym, tf):
    p_c = DATA / sym / f"{tf}.csv"
    p_m = MAP / sym / f"{tf}_segments_mapped.csv"
    if not (p_c.exists() and p_m.exists()):
        return dict(symbol=sym, tf=tf, ok=False, msg="missing files")

    c = _read_candles(p_c)
    m = pd.read_csv(p_m)
    if m.empty or c.empty:
        return dict(symbol=sym, tf=tf, ok=False, msg="empty")


    bad_range = ((m["start_bar_index"] < 0) | (m["end_bar_index"] >= len(c))).sum()

    bad_order = (m["start_bar_index"] > m["end_bar_index"]).sum()

    end_ts = c.loc[m["end_bar_index"].astype(int), "timestamp"].to_numpy()
    seg_end = pd.to_datetime(m["timestamp_end"], utc=True).to_numpy()
    bad_future = (end_ts > seg_end).sum()

    seg_len = (m["end_bar_index"] - m["start_bar_index"] + 1).clip(lower=1)
    cover_ratio = float(seg_len.mean() / max(1, len(c)))

    return dict(
        symbol=sym, tf=tf, ok=(bad_range==0 and bad_order==0 and bad_future==0),
        bad_range=int(bad_range), bad_order=int(bad_order), bad_future=int(bad_future),
        n_segments=int(len(m)), n_candles=int(len(c)), avg_span=float(seg_len.mean()),
        cover_ratio=cover_ratio
    )

rows = []
for sym in symbols:
    for tf in tfs:
        rows.append(_quick_checks(sym, tf))

chk = pd.DataFrame(rows)
print("自检摘要：")
display(chk.sort_values(["symbol","tf"]))


自检摘要：


symbol   tf    ok  bad_range  bad_order  bad_future  n_segments  \
1  BTCUSDT  15m  True          0          0           0         116   
3  BTCUSDT   1h  True          0          0           0          64   
2  BTCUSDT  30m  True          0          0           0         105   
4  BTCUSDT   4h  True          0          0           0          17   
0  BTCUSDT   5m  True          0          0           0         112   
6  ETHUSDT  15m  True          0          0           0         256   
8  ETHUSDT   1h  True          0          0           0          78   
7  ETHUSDT  30m  True          0          0           0         142   
9  ETHUSDT   4h  True          0          0           0          20   
5  ETHUSDT   5m  True          0          0           0         408   

   n_candles    avg_span  cover_ratio  
1      19200   85.905172     0.004474  
3       4800   40.750000     0.008490  
2       9600   49.933333     0.005201  
4       1200   34.058824     0.028382  
0      57600  239.035714     0.004150  
6      19200   43.531250     0.002267  
8       4800   37.089744     0.007727  
7       9600   35.105634     0.003657  
9       1200   34.300000     0.028583  
5      57600   79.700980     0.001384